In [3]:
using PrettyTables

function RK4(f, x0, y0, N, h)
    #Use a 4th order Runge-Kutta method to solve y'=f(x,y), with y(x0)=y0 using N steps of size h.
    #Returns an array of (x_i, w_i) pairs.
    approx = [x0 y0]
    x = x0
    w = y0
    for i in 1:N
        k1 = f(x, w)
        k2 = f(x + h / 2, w + h * k1 / 2)
        k3 = f(x + h / 2, w + h * k2 / 2)
        k4 = f(x + h, w + h * k3)
        w = w + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
        x += h
        approx = vcat(approx, [x w]) #Add a new row to the matrix of approximation points
    end
    return approx
end

function AB4(f, x0, y0, N, h)
    x1 = x0 + h
    x2 = x1 + h
    x3 = x2 + h
    w0 = y0
    RK_approx = RK4(f, x0, y0, 3, h)
    w1 = RK_approx[2, 2]
    w2 = RK_approx[3, 2]
    w3 = RK_approx[4, 2]
    approx = [x0 w0; x1 w1; x2 w2; x3 w3]
    for i in 1:N-3
        x4 = x3 + h
        w4 = w3 + h / 24 * (55 * f(x3, w3) - 59 * f(x2, w2) + 37 * f(x1, w1) - 9 * f(x0, w0))
        approx = vcat(approx, [x4 w4])
        x0 = x1
        x1 = x2
        x2 = x3
        x3 = x4
        w0 = w1
        w1 = w2
        w2 = w3
        w3 = w4
    end
    return approx
end

function AdamsPredictorCorrector(f, x0, y0, N, h)
    #Predictor corrector method combining 4 step Adams-Bashforth with 3 step Adams-Moulton
    x1 = x0 + h
    x2 = x1 + h
    x3 = x2 + h
    w0 = y0
    RK_approx = RK4(f, x0, y0, 3, h) #Initialize method with RK4
    w1 = RK_approx[2, 2]
    w2 = RK_approx[3, 2]
    w3 = RK_approx[4, 2]
    approx = [x0 w0; x1 w1; x2 w2; x3 w3]
    for i in 1:N-3
        x4 = x3 + h
        #Store these for re-use in AM3
        f1 = f(x1, w1)
        f2 = f(x2, w2)
        f3 = f(x3, w3)
        #Predictor step
        w4p = w3 + h / 24 * (55 * f3 - 59 * f2 + 37 * f1 - 9 * f(x0, w0))
        #Corrector Step     
        w4 = w3 + h / 24 * (9 * f(x4, w4p) + 19 * f3 - 5 * f2 + f1) 
        approx = vcat(approx, [x4 w4])
        #Update starting values
        x0 = x1
        x1 = x2
        x2 = x3
        x3 = x4
        w0 = w1
        w1 = w2
        w2 = w3
        w3 = w4
    end
    return approx
end

AdamsPredictorCorrector (generic function with 1 method)

In [11]:
#Example comparing Adams-Bashforth, and Predictor-Corrector methods to solve y' = y, y(0) = 1.
f(x, y) = y
exact(x) = exp(x)
a = 0
b = 1
y0 = 1
errors = []
for i in 1:4
    N = 10^i
    h = (b - a) / N
    println("Adams-Bashforth Time: ")
    @time begin #We'll compare times for the comparable AB and AM methods
    approx = AB4(f, a, y0, N, h)
    y_ab4 = approx[N + 1, 2]
    e_ab4 = abs(y_ab4 - exact(b))
    end
    println("Predictor-Corrector Time: ")
    @time begin
    approx = AdamsPredictorCorrector(f, a, y0, N, h)
    y_pc = approx[N + 1, 2]
    e_pc = abs(y_pc - exact(b))
    end
    if i == 1
        errors = [h e_ab4 e_pc]
    else
        errors = vcat(errors, [h e_ab4 e_pc])
    end
end
pretty_table(errors, column_labels = ["h", "AB4", "PC"])

Adams-Bashforth Time: 
  0.085162 seconds (41.07 k allocations: 2.644 MiB, 99.92% compilation time: 93% of which was recompilation)
Predictor-Corrector Time: 
  0.067824 seconds (26.22 k allocations: 1.690 MiB, 99.94% compilation time: 100% of which was recompilation)
Adams-Bashforth Time: 
  0.000052 seconds (227 allocations: 98.938 KiB)
Predictor-Corrector Time: 
  0.000044 seconds (227 allocations: 98.938 KiB)
Adams-Bashforth Time: 
  0.001068 seconds (2.03 k allocations: 7.869 MiB)
Predictor-Corrector Time: 
  0.004408 seconds (2.03 k allocations: 7.869 MiB)
Adams-Bashforth Time: 
  0.424963 seconds (29.01 k allocations: 764.692 MiB, 13.44% gc time)
Predictor-Corrector Time: 
  0.405020 seconds (29.01 k allocations: 764.692 MiB, 12.67% gc time)
┌────────┬─────────────┬─────────────┐
│      h │         AB4 │          PC │
├────────┼─────────────┼─────────────┤
│    0.1 │  5.73893e-5 │  1.79029e-6 │
│   0.01 │  9.05699e-9 │ 6.47201e-10 │
│  0.001 │ 9.41913e-13 │ 6.66134e-14 │
│ 0.000